# Update run_setup yaml with specific setting combinations

Task is to selectively update specific settings and inputs. To ensure more consistency and flexiblity, we place key settings in dictionaries.

In [1]:
import pandas as pd
import yaml
import pathlib
import os

In [3]:
HOME_DIR = pathlib.Path.home()

# in my (AO) case, M:\urban_modeling is mounted to /Volumes/Data/Models/urban_modeling
M_DRIVE = pathlib.Path("/Volumes/Data/Models") if os.name != "nt" else "M:/"

BOX_DIR = HOME_DIR / 'Library/CloudStorage/Box-Box'

In [10]:
np_input_path = M_DRIVE / 'urban_modeling/baus/PBA50Plus/staging_yaml/PBA50Plus_NoProject_TEST_NP_XX.yaml'
np_input_path = M_DRIVE / 'urban_modeling/baus/PBA50Plus/PBA50Plus_NoProject/PBA50Plus_NoProject_v35b/run_setup_PBA50Plus_NoProject_v35b.yaml'
np_input_path = M_DRIVE / 'urban_modeling/baus/PBA50Plus/PBA50Plus_NoProject/PBA50Plus_NoProject_v35d/run_setup_PBA50Plus_NoProject_v35d.yaml'

# np_output_path = '/Volumes/Data/Models/urban_modeling/baus/PBA50Plus/staging_yaml/PBA50Plus_NoProject_TEST_NP_ZZ.yaml'
dbp_input_path = M_DRIVE / 'urban_modeling/baus/PBA50Plus/PBA50Plus_DraftBlueprint/PBA50Plus_Draft_Blueprint_v8_znupd_nodevfix/run_setup_PBA50Plus_Draft_Blueprint_v8_znupd_nodevfix.yaml'

fbp_input_path = M_DRIVE / 'urban_modeling/baus/PBA50Plus/PBA50Plus_FinalBlueprint/PBA50Plus_Final_Blueprint_v55b/run_setup_PBA50Plus_Final_Blueprint_v55b.yaml'

In [11]:
# input_path = output_path
output_base_path = M_DRIVE / 'urban_modeling/baus/PBA50Plus/staging_yaml_v5'
# output_base_path = pathlib.Path('/Users/aolsen/temp/baus/staging_yaml_v2')
output_base_path.mkdir(exist_ok=True)

In [12]:
def load_yaml(yaml_path):
    with open(yaml_path, "r") as file:
        return yaml.safe_load(file)

In [13]:
def save_yaml(yaml_path, setup_dict):
    with open(yaml_path, "w") as file:
        file.write(yaml.dump(setup_dict))

In [14]:
#setup_dict_FBP = load_yaml(fbp_input_path)
setup_dict_DBP = load_yaml(dbp_input_path)
setup_dict_NP = load_yaml(np_input_path)
#pd.Series(setup_dict_DBP)[pd.Series(setup_dict_DBP).index.str.contains('housing_bond_strategy')].to_dict()

# Step 1: Set settings in a dict so we can easily update the core settings a la carte

These can be pulled in flexibly by a separate "scenario" dict in step 2, which then is used to build the actual yaml.

In [15]:
stock_settings = {
    'EC1': {'NP': {'household_controls_file': 'household_controls_PBA50Plus_NP.csv'},
            'DBP': {'household_controls_file': 'household_controls_PBA50Plus_DBP_UBI2030.csv'},
            'FBP': {'household_controls_file': 'household_controls_PBA50Plus_DBP_UBI2030.csv'},

            },
    'exog': {
        'FBP': {'exog_sqft_per_job_adj_file': 'sqft_per_job_adjusters_costar_qcew_timevarying_base_2023_0p85_reduction_dec2024_sd_2035.csv'},
        'DBP': {'exog_sqft_per_job_adj_file': 'sqft_per_job_adjusters_exogenous.csv'},
        'NP': {'exog_sqft_per_job_adj_file': 'sqft_per_job_adjusters_costar_qcew_timevarying_base_2023_0p85_reduction_dec2024_sd_2035.csv'}
    },
    'base_zoning': {
        'FBP': {'zoning_file': 'zoning_parcels_2024-10-14.csv', 'zoning_lookup_file': 'zoning_lookup_2024-10-14.csv'},
        'DBP': {'zoning_file': 'zoning_parcels_2024-04-30.csv', 'zoning_lookup_file': 'zoning_lookup_2024-04-30.csv'},
        'NP': {'zoning_file': 'zoning_parcels_2024-10-14.csv', 'zoning_lookup_file': 'zoning_lookup_2024-10-14.csv'},
    },

    'pipeline': {
        'FBP_ALT': {'development_pipeline_file': '2025_0331_2225_development_projects_PBA50p_NP.csv'},
        'FBP': {'development_pipeline_file': '2024_1026_2234_development_projects_PBA50p_NP.csv'},
        'DBP': {'development_pipeline_file': 'development_pipeline_NP_2024-03-08.csv'},
        'NP': {'development_pipeline_file': 'development_pipeline_NP_2024-03-08.csv'}
    },
    'EC5': {
        'FBP': {'ec5_spec_file': 'elcm_ec5_loose.yaml', 'run_jobs_to_transit_strategy_elcm': True},
        'DBP': {'ec5_spec_file': 'elcm_ec5_loose.yaml', 'run_jobs_to_transit_strategy_elcm': True},

    },
    'H1': {
        'FBP': {'run_renter_protections_strategy': True},
        'DBP': {'run_renter_protections_strategy': True},
        'NP': {'run_renter_protections_strategy': False}
    },

    'H2': {
        'FBP': {'run_housing_preservation_strategy': True,
                'preservation_file': 'preservation_PBA50Plus_FBP_MAR25.yaml'},
        'DBP': {'run_housing_preservation_strategy': True,
                'preservation_file': 'preservation_PBA50Plus_DBP.yaml'},
        'NP': {'run_housing_preservation_strategy': False
               },
    },
    'H4': {
        'FBP': {'run_housing_bond_strategy': True,
                'account_strategies_file': 'account_strategies_PBA50Plus_FBP.yaml', 'run_housing_bond_strategy': True,
                'run_alameda_housing_bond_strategy': True,
                'run_contra_costa_housing_bond_strategy': True,
                'run_marin_housing_bond_strategy': True,
                'run_napa_housing_bond_strategy': True,
                'run_san_francisco_housing_bond_strategy': True,
                'run_san_mateo_housing_bond_strategy': True,
                'run_santa_clara_housing_bond_strategy': True,
                'run_solano_housing_bond_strategy': True,
                'run_sonoma_housing_bond_strategy': True},
        'DBP': {'run_housing_bond_strategy': True,
                'account_strategies_file': 'account_strategies_PBA50Plus_DBP.yaml', 'run_housing_bond_strategy': True,
                'run_alameda_housing_bond_strategy': True,
                'run_contra_costa_housing_bond_strategy': True,
                'run_marin_housing_bond_strategy': True,
                'run_napa_housing_bond_strategy': True,
                'run_san_francisco_housing_bond_strategy': True,
                'run_san_mateo_housing_bond_strategy': True,
                'run_santa_clara_housing_bond_strategy': True,
                'run_solano_housing_bond_strategy': True,
                'run_sonoma_housing_bond_strategy': True},
        'NP': {'run_housing_bond_strategy': False
               },
    },

    'H5': {
        'FBP': {'run_inclusionary_strategy': True,
                'inclusionary_strategy_file': 'inclusionary_strategy_PBA50Plus_FBP_MAR14_25.yaml'},
        'DBP': {'run_inclusionary_strategy': True,
                'inclusionary_strategy_file': 'inclusionary_strategy_PBA50Plus_DBP.yaml'},
        'NP': {'run_inclusionary_strategy': False
               },
    },
    'strategy_pipeline_all': {
        'FBP': {'dev_pipeline_strategies': ['EC2_development_pipeline_entries.csv',
                                            'EC6_development_pipeline_entries.csv',
                                            'H6_H8_development_pipeline_entries_FBP_MAR.csv',
                                            'EC5_development_pipeline_anchor_buildings_FBP_v4.csv']},
        'DBP': {'dev_pipeline_strategies': ['EC2_development_pipeline_entries.csv',
                                            'EC6_development_pipeline_entries.csv',
                                            'H6_H8_development_pipeline_entries_FBP_MAR.csv',
                                            # 'EC5_development_pipeline_anchor_buildings_FBP_APR1.csv',
                                            'EC5_development_pipeline_anchor_buildings_FBP_v4.csv']},

    },
    'strategy_pipeline_EC5': {
        'FBP': {'dev_pipeline_strategies': [  # 'EC5_development_pipeline_anchor_buildings_FBP_APR1.csv',
            'EC5_development_pipeline_anchor_buildings_FBP_v4.csv']},
        'DBP': {'dev_pipeline_strategies': [  # 'EC5_development_pipeline_anchor_buildings_FBP_APR1.csv',
            'EC5_development_pipeline_anchor_buildings_FBP_v4.csv']},

    },
    'strategy_pipeline_no_EC5': {
        'FBP': {'dev_pipeline_strategies': ['EC2_development_pipeline_entries.csv',
                                            'EC6_development_pipeline_entries.csv',
                                            'H6_H8_development_pipeline_entries_FBP_MAR.csv'
                                            ]},
        'DBP': {'dev_pipeline_strategies': ['EC2_development_pipeline_entries.csv',
                                            'EC6_development_pipeline_entries.csv',
                                            'H6_H8_development_pipeline_entries_FBP_MAR.csv',
                                            ]},

    },

    'SLR': {
        'FBP': {'slr_inundation_file': "urbansim_slr_MAR2025.csv",
                'slr_progression_file': "slr_progression_PBA50Plus.csv",
                'run_slr': True},
        'DBP': {'slr_inundation_file': 'slr_parcel_inundation_PBA50Plus_DBP.csv',
                'slr_progression_file': "slr_progression_PBA50Plus.csv", 'run_slr': True},
        'NP': {'slr_inundation_file': 'urbansim_slr_no_project_MAR2025.csv',
               'slr_progression_file': "slr_progression_PBA50Plus.csv", 'run_slr': True

               }
    },
    'profit_adj': {
        'FBP': {'profit_adjustment_strategies_file': "profit_adjustment_strategies_PBA50_FBP_newgg_APR_2025.yaml",
                'run_reduce_housing_costs_tier1_strategy': True, 'run_reduce_housing_costs_tier2_strategy': True,
                'run_reduce_housing_costs_tier3_strategy': True
                },
        'DBP': {'profit_adjustment_strategies_file': "profit_adjustment_strategies_PBA50_FBP.yaml",
                'run_reduce_housing_costs_tier1_strategy': True, 'run_reduce_housing_costs_tier2_strategy': True,
                'run_reduce_housing_costs_tier3_strategy': True
                },
        'NP': {'profit_adjustment_strategies_file': "profit_adjustment_strategies_PBA50_NP.yaml",
               'run_reduce_housing_costs_tier1_strategy': False, 'run_reduce_housing_costs_tier2_strategy': False,
               'run_reduce_housing_costs_tier3_strategy': False
               }
    },

    'parcels_and_zoning': {
        'FBP': {'parcels_geography_file': 'fbp_urbansim_parcel_classes_ot50pct.csv',
                'parcels_geography_cols': ['gg_id',
                                           'exd_id',
                                           'pda_id',
                                           'tra_id',
                                           'hra_id',
                                           'dis_id',
                                           'ppa_id',
                                           'ugb_id'],
                'zoning_mods_file': 'zoning_mods_PBA50Plus_FBP_MAR14_2025.csv',
                'zoningmodcat_cols': ['gg_id',
                                      'exd_id',
                                      'tra_id',
                                      'hra_id',
                                      'ppa_id',
                                      'ugb_id']},
        'FBP_ALT': {'parcels_geography_file': 'fbp_urbansim_parcel_classes_ot50pct.csv',
                    'parcels_geography_cols': ['gg_id',
                                               'exd_id',
                                               'pda_id',
                                               'tra_id',
                                               'hra_id',
                                               'dis_id',
                                               'ppa_id',
                                               'ugb_id'],
                    'zoning_mods_file': 'zoning_mods_PBA50Plus_FBP_MAR24_2025.csv',
                    'zoningmodcat_cols': ['gg_id',
                                          'exd_id',
                                          'tra_id',
                                          'hra_id',
                                          'ppa_id',
                                          'ugb_id']},
        'DBP': {'parcels_geography_file': 'parcels_geography_2024_02_14.csv',
                'parcels_geography_cols': ['gg_id',
                                           'exd_id',
                                           'pda_id',
                                           'tra_id',
                                           'hra_id',
                                           'epc_id',
                                           'dis_id',
                                           'ppa_id',
                                           'ugb_id'],
                'zoning_mods_file': 'zoning_mods_PBA50Plus_DBP.csv',
                'zoningmodcat_cols': ['gg_id',
                                      'exd_id',
                                      'tra_id',
                                      'hra_id',
                                      'ppa_id',
                                      'ugb_id']},
        'NP': {'parcels_geography_file': 'parcels_geography_NP_2024_04_29.csv',
               'parcels_geography_cols': ['gg_id',
                                          'exd_id',
                                          'pda_id',
                                          'tra_id',
                                          'hra_id',
                                          'epc_id',
                                          'dis_id',
                                          'ppa_id',
                                          'ugb_id'],
               'zoning_mods_file': 'zoning_mods_PBA50Plus_NP_exp.csv',
               'zoningmodcat_cols': ['gg_id',
                                     'exd_id',
                                     'tra_id',
                                     'hra_id',
                                     'ppa_id',
                                     'ugb_id']}
    }
}

# Step 2: Define scenarios by pulling together bundles of settings

In [16]:
scenario_desc = {  # NP to FBP sequence
    'NP00': 'NP + EC1',
    'NP01': 'NP + zoningmods (using FBP GG parcel assignment) + EC1',
    'NP02': 'NP + zoningmods + EC1 + other housing strategies based on GG (e.g. inclusionary, preservation, subsidy) + NP SLR',
    'NP03': 'NP + zoningmods + EC1 + other housing strategies + other economy strategies (EC5) + NP SLR',
    'NP04': 'NP + zoningmods + EC1 + other housing strategies + other economy strategies (EC5) + strategy projects + NP SLR',
    'NP05': 'NP + zoningmods + EC1 + other housing strategies + other economy strategies (EC5) + strategy projects + FBP SLR',
    # 'NP05b': 'NP + updated base zoning + Pipe Alt + zoningmods + other housing strategies + other economy strategies (EC5) + strategy projects + SLR',

    # variants of np04
    'NP05b': 'NP + DBP Exog + zoningmods + EC1 + other housing strategies + other economy strategies (EC5) + strategy projects + SLR',
    'NP05c': 'NP + DBP base zoning + zoningmods + EC1 + other housing strategies + other economy strategies (EC5) + strategy projects + SLR',
    'NP05d': 'NP + FBP pipeline + zoningmods + EC1 + other housing strategies + other economy strategies (EC5) + strategy projects + SLR',


    # DBP sequence
    'FBP00': 'DBP + exogenous of FBP',
    'FBP01': "DBP + exogenous of FBP + other strategies that are based on GG: switch to FBP GG schema and apply it to those strategies, while keeping the DBP-equivalent zoningmods using FBP GG (Mark's version)",
    'FBP02': 'DBP + exogenous of FBP + other strategies that are based on GG + FBP zoningmods (the spiky one)',
    'FBP03': 'DBP + exogenous of FBP + other strategies that are based on GG + FBP zoningmods + strategy projects',
    'FBP04': 'DBP + exogenous of FBP + other strategies that are based on GG + FBP zoningmods + strategy projects + EC5 change',
    'FBP05': 'DBP + exogenous of FBP + other strategies that are based on GG + FBP zoningmods + strategy projects + EC5 change + FBP SLR',

    # variants of fbp05
    'FBP05b': 'DBP + DBP Exog + other strategies that are based on GG + FBP zoningmods + strategy projects + EC5 change + FBP SLR',
    'FBP05c': 'DBP + FBP Exog + DBP base zoning +  other strategies that are based on GG + FBP zoningmods + strategy projects + EC5 change + FBP SLR',
    'FBP05d': 'DBP + FBP Exog + FBP pipeline +  other strategies that are based on GG + FBP zoningmods + strategy projects + EC5 change + FBP SLR'}



In [17]:
scenario_mapping = {  # starting with NP, then adding piece-wise
    'NP_to_FBP': {'NP00': {'EC1': 'FBP'},


                  'NP01': {'base_zoning': 'NP', 'exog': 'FBP', 'parcels_and_zoning': 'FBP',
                           'EC1': 'FBP'},

                  'NP02': {'base_zoning': 'NP', 'EC1': 'FBP', 'exog': 'FBP',
                           'parcels_and_zoning': 'FBP', 'H2': 'FBP', 'H4': 'FBP',
                           'H5': 'FBP', 'profit_adj': 'FBP'},

                  'NP03': {'base_zoning': 'NP', 'EC1': 'FBP', 'exog': 'FBP',
                           'parcels_and_zoning': 'FBP', 'H2': 'FBP', 'H4': 'FBP', 'H5': 'FBP',
                           'EC5': 'FBP', 'strategy_pipeline_EC5': 'FBP', 'profit_adj': 'FBP'},

                  'NP04': {'base_zoning': 'NP', 'EC1': 'FBP', 'exog': 'FBP',
                           'parcels_and_zoning': 'FBP', 'H2': 'FBP', 'H4': 'FBP', 'H5': 'FBP',
                           'EC5': 'FBP', 'strategy_pipeline_all': 'FBP', 'profit_adj': 'FBP'},

                  'NP05': {'base_zoning': 'NP', 'EC1': 'FBP', 'exog': 'FBP',
                           'parcels_and_zoning': 'FBP', 'H2': 'FBP', 'H4': 'FBP', 'H5': 'FBP',
                           'EC5': 'FBP', 'strategy_pipeline_all': 'FBP', 'SLR': 'FBP', 'profit_adj': 'FBP'},

                  'NP05b': {'base_zoning': 'NP', 'EC1': 'FBP', 'exog': 'DBP',
                            'parcels_and_zoning': 'FBP', 'H2': 'FBP', 'H4': 'FBP', 'H5': 'FBP',
                            'EC5': 'FBP', 'strategy_pipeline_all': 'FBP',
                            'SLR': 'FBP', 'profit_adj': 'FBP'},

                  'NP05c': {'base_zoning': 'DBP', 'EC1': 'FBP', 'exog': 'FBP',
                            'parcels_and_zoning': 'FBP', 'H2': 'FBP', 'H4': 'FBP', 'H5': 'FBP',
                            'EC5': 'FBP', 'strategy_pipeline_all': 'FBP', 'SLR': 'FBP',
                            'profit_adj': 'FBP'},
                  'NP05d': {'base_zoning': 'NP', 'EC1': 'FBP', 'exog': 'FBP',
                            'pipeline': 'FBP', 'parcels_and_zoning': 'FBP', 'H2': 'FBP',
                            'H4': 'FBP', 'H5': 'FBP', 'EC5': 'FBP',
                            'strategy_pipeline_all': 'FBP', 'SLR': 'FBP', 'profit_adj': 'FBP'},

                  },

    # starting with DBP, then adding piece wise
    'DBP_to_FBP': {
        'FBP00': {'base_zoning': 'FBP', 'exog': 'FBP', 'profit_adj': 'FBP'},

        'FBP01': {'base_zoning': 'FBP', 'exog': 'FBP', 'profit_adj': 'FBP',
                  'parcels_and_zoning': 'FBP_ALT', 'H2': 'FBP', 'H4': 'FBP', 'H5': 'FBP'},

        'FBP02': {'base_zoning': 'FBP', 'exog': 'FBP', 'profit_adj': 'FBP',
                  'parcels_and_zoning': 'FBP', 'H2': 'FBP', 'H4': 'FBP', 'H5': 'FBP'},

        'FBP03': {'base_zoning': 'FBP', 'exog': 'FBP', 'profit_adj': 'FBP',
                  'parcels_and_zoning':     'FBP', 'H2': 'FBP', 'H4': 'FBP', 'H5': 'FBP',
                  'strategy_pipeline_no_EC5': 'FBP'},

        'FBP04': {'base_zoning': 'FBP', 'exog': 'FBP', 'profit_adj': 'FBP',
                  'parcels_and_zoning':     'FBP', 'H2': 'FBP', 'H4': 'FBP', 'H5': 'FBP',
                  'strategy_pipeline_all': 'FBP', 'EC5': 'FBP'},

        'FBP05': {'base_zoning': 'FBP', 'exog': 'FBP', 'profit_adj': 'FBP',
                  'parcels_and_zoning':     'FBP', 'H2': 'FBP', 'H4': 'FBP', 'H5': 'FBP',
                  'strategy_pipeline_all': 'FBP', 'EC5': 'FBP', 'SLR': 'FBP'},

        'FBP05b': {'base_zoning': 'FBP', 'exog': 'DBP', 'profit_adj': 'FBP',
                   'parcels_and_zoning':     'FBP', 'H2': 'FBP', 'H4': 'FBP', 'H5': 'FBP',
                   'strategy_pipeline_all': 'FBP', 'EC5': 'FBP', 'SLR': 'FBP'},

        'FBP05c': {'base_zoning': 'DBP', 'exog': 'FBP',  'profit_adj': 'FBP',
                   'parcels_and_zoning':     'FBP', 'H2': 'FBP', 'H4': 'FBP', 'H5': 'FBP',
                   'strategy_pipeline_all': 'FBP', 'EC5': 'FBP', 'SLR': 'FBP'},

        'FBP05d': {'base_zoning': 'FBP', 'exog': 'FBP',  'pipeline': 'FBP',
                   'profit_adj': 'FBP',
                   'parcels_and_zoning': 'FBP', 'H2': 'FBP', 'H4': 'FBP', 'H5': 'FBP',
                   'strategy_pipeline_all': 'FBP', 'EC5': 'FBP', 'SLR': 'FBP'},

        #         'FBP06b': {'exog': 'FBP', 'profit_adj': 'FBP', 'parcels_and_zoning':    'FBP', 'H2': 'FBP', 'H4': 'FBP', 'H5': 'FBP', 'H6_H8_pipe': 'FBP', 'EC6_pipe': 'FBP', 'EC5_pipe': 'FBP', 'EC5': 'FBP', 'SLR': 'FBP',
        #                    'base_zoning': 'FBP'},
        #         'FBP06': {'exog': 'FBP', 'profit_adj': 'FBP', 'parcels_and_zoning': 'FBP', 'H2': 'FBP', 'H4': 'FBP', 'H5': 'FBP', 'H6_H8_pipe': 'FBP', 'EC6_pipe': 'FBP', 'EC5_pipe': 'FBP', 'EC5': 'FBP', 'SLR': 'FBP',
        #                   'base_zoning': 'FBP', 'pipeline': 'FBP_ALT'}

    }

}

In [18]:
def sort_nested_dict(input_dict):
    """
    Recursively sorts the keys of a nested dictionary alphabetically.
    Returns a new dictionary with the sorted keys.

    """
    sorted_dict = {}
    for key in sorted(input_dict.keys()):
        value = input_dict[key]
        if isinstance(value, dict):
            sorted_dict[key] = sort_nested_dict(value)
        else:
            sorted_dict[key] = value
    return sorted_dict


scenario_mapping_sorted = sort_nested_dict(scenario_mapping)

## Which series has which policies?
NaN just means no explicit strategy selection was made for that strategy. That means it goes with what the starting configuration has.

In [19]:
pd.Series(scenario_mapping['NP_to_FBP']).apply(pd.Series).stack().sort_index()

NP00   EC1                      FBP
NP01   EC1                      FBP
       base_zoning               NP
       exog                     FBP
       parcels_and_zoning       FBP
                               ... 
NP05d  exog                     FBP
       parcels_and_zoning       FBP
       pipeline                 FBP
       profit_adj               FBP
       strategy_pipeline_all    FBP
Length: 78, dtype: object

# Generate the yaml updates, starting from a relevant base dict

In [20]:
for proj, scens in scenario_mapping.items():
    print(proj)

    if proj == 'NP_to_FBP':
        this_setup_dict = setup_dict_NP.copy()
    else:
        this_setup_dict = setup_dict_DBP.copy()

    this_setup_dict['viz_dir'] = 'M:/urban_modeling/baus/PBA50Plus/BAUS_Visualizer_PBA50Plus_Files/PRODUCTION'
    this_setup_dict['outputs_dir'] = 'M:/urban_modeling/baus/PBA50Plus/PBA50Plus_FinalBlueprint'
    # this_setup_dict['outputs_dir'] = 'E:/baus_runs/contrast'
    this_setup_dict['run_slr_summaries'] = False

    for k, subdict in scens.items():
        # print(k)
        print('\t', subdict)
        annot = []
        for k1, v1 in subdict.items():
            print('\t'*2, k1)
            print('\t'*2, v1)

            this_setup_dict.update(stock_settings[k1][v1])
            annot.append(yaml.dump(stock_settings[k1][v1]))
        this_setup_dict['run_name'] = f'PBA50Plus_CNTRST_{proj}_{k}_v5'
        this_setup_dict['annotation'] = [scenario_desc[k]]

        save_yaml(output_base_path /
                  f'run_setup_test_{k}.yaml', this_setup_dict)

NP_to_FBP
	 {'EC1': 'FBP'}
		 EC1
		 FBP
	 {'base_zoning': 'NP', 'exog': 'FBP', 'parcels_and_zoning': 'FBP', 'EC1': 'FBP'}
		 base_zoning
		 NP
		 exog
		 FBP
		 parcels_and_zoning
		 FBP
		 EC1
		 FBP
	 {'base_zoning': 'NP', 'EC1': 'FBP', 'exog': 'FBP', 'parcels_and_zoning': 'FBP', 'H2': 'FBP', 'H4': 'FBP', 'H5': 'FBP', 'profit_adj': 'FBP'}
		 base_zoning
		 NP
		 EC1
		 FBP
		 exog
		 FBP
		 parcels_and_zoning
		 FBP
		 H2
		 FBP
		 H4
		 FBP
		 H5
		 FBP
		 profit_adj
		 FBP
	 {'base_zoning': 'NP', 'EC1': 'FBP', 'exog': 'FBP', 'parcels_and_zoning': 'FBP', 'H2': 'FBP', 'H4': 'FBP', 'H5': 'FBP', 'EC5': 'FBP', 'strategy_pipeline_EC5': 'FBP', 'profit_adj': 'FBP'}
		 base_zoning
		 NP
		 EC1
		 FBP
		 exog
		 FBP
		 parcels_and_zoning
		 FBP
		 H2
		 FBP
		 H4
		 FBP
		 H5
		 FBP
		 EC5
		 FBP
		 strategy_pipeline_EC5
		 FBP
		 profit_adj
		 FBP
	 {'base_zoning': 'NP', 'EC1': 'FBP', 'exog': 'FBP', 'parcels_and_zoning': 'FBP', 'H2': 'FBP', 'H4': 'FBP', 'H5': 'FBP', 'EC5': 'FBP', 'strategy

# Generate the description for asana

In [315]:
table_for_asana_newline = pd.Series(scenario_desc).str.replace(
    '+', '\n+')
table_for_asana = pd.DataFrame(data={'desc': table_for_asana_newline, 'category': table_for_asana_newline.index.str.extract(
    '(NP|FBP)', expand=False).map({'NP': 'Draft Blueprint → Final Blueprint', 'FBP': 'NoProject → Final Blueprint'})})
table_for_asana.to_clipboard()
table_for_asana.reset_index()

,index,desc,category
0,NP00,NP \n+ EC1,Draft Blueprint → Final Blueprint
1,NP01,NP \n+ zoningmods (using FBP GG parcel assignm...,Draft Blueprint → Final Blueprint
2,NP02,NP \n+ zoningmods \n+ EC1 \n+ other housing st...,Draft Blueprint → Final Blueprint
3,NP03,NP \n+ zoningmods \n+ EC1 \n+ other housing st...,Draft Blueprint → Final Blueprint
4,NP04,NP \n+ zoningmods \n+ EC1 \n+ other housing st...,Draft Blueprint → Final Blueprint
5,NP05,NP \n+ zoningmods \n+ EC1 \n+ other housing st...,Draft Blueprint → Final Blueprint
6,NP05b,NP \n+ DBP Exog \n+ zoningmods \n+ EC1 \n+ oth...,Draft Blueprint → Final Blueprint
7,NP05c,NP \n+ DBP base zoning \n+ zoningmods \n+ EC1 ...,Draft Blueprint → Final Blueprint
8,NP05d,NP \n+ FBP pipeline \n+ zoningmods \n+ EC1 \n+...,Draft Blueprint → Final Blueprint
9,FBP00,DBP \n+ exogenous of FBP,NoProject → Final Blueprint


## try automated approach

In [316]:
import pprint
description_templates = {
    'base_zoning': "{value} base zoning",
    'exog': "{value} exogenous data",
    'parcels_and_zoning': {
        'FBP_ALT': "zoning mods (FBP_ALT)",
        'FBP': "zoning mods (FBP)",
        'DBP': "zoning mods (DBP)",
        'NP': "zoning mods (NP)",
        'default': "zoning mods ({value})"
    },
    'pipeline': "development pipeline ({value})",
    'EC1': "EC1",
    'EC5': "EC5",
    'H1': "H1 ({value})",
    'H2': "H2 ({value})",
    'H4': "H4 ({value})",
    'H5': "H5 ({value})",
    'SLR': "SLR EN1 ({value})",
    'profit_adj': "Profit Adjustment ({value})",
    'strategy_pipeline_all': "Strategy Pipeline (all)",
    'strategy_pipeline_EC5': "Strategy Pipeline (EC5 only)",
    'strategy_pipeline_no_EC5': "Strategy Pipeline (no EC5)",
    'default': "{key} ({value})"  # Catch-all
}


def generate_scenario_description_with_templates(scenario_config, templates):
    # try to use a template so we can generate based on what is actually in the dicts
    # useful to compare against YW's "target" descriptions
    description_parts = []
    for key, value in scenario_config.items():
        if key in templates:
            template = templates[key]
            if isinstance(template, dict):
                description_parts.append(template.get(
                    value, template.get('default', f"{key}: {value}")))
            else:
                description_parts.append(template.format(value=value))
        else:
            description_parts.append(
                templates['default'].format(key=key, value=value))
    return "\n+ ".join(description_parts)


generated_desc_from_templates = {}
for outer_key, inner_dict in scenario_mapping.items():
    generated_desc_from_templates[outer_key] = {}
    for scenario_id, config in inner_dict.items():
        generated_desc_from_templates[outer_key][scenario_id] = generate_scenario_description_with_templates(
            config, description_templates)

autodesc = generated_desc_from_templates
autodesc

{'NP_to_FBP': {'NP00': 'EC1',
  'NP01': 'NP base zoning\n+ FBP exogenous data\n+ zoning mods (FBP)\n+ EC1',
  'NP02': 'NP base zoning\n+ EC1\n+ FBP exogenous data\n+ zoning mods (FBP)\n+ H2 (FBP)\n+ H4 (FBP)\n+ H5 (FBP)\n+ Profit Adjustment (FBP)',
  'NP03': 'NP base zoning\n+ EC1\n+ FBP exogenous data\n+ zoning mods (FBP)\n+ H2 (FBP)\n+ H4 (FBP)\n+ H5 (FBP)\n+ EC5\n+ Strategy Pipeline (EC5 only)\n+ Profit Adjustment (FBP)',
  'NP04': 'NP base zoning\n+ EC1\n+ FBP exogenous data\n+ zoning mods (FBP)\n+ H2 (FBP)\n+ H4 (FBP)\n+ H5 (FBP)\n+ EC5\n+ Strategy Pipeline (all)\n+ Profit Adjustment (FBP)',
  'NP05': 'NP base zoning\n+ EC1\n+ FBP exogenous data\n+ zoning mods (FBP)\n+ H2 (FBP)\n+ H4 (FBP)\n+ H5 (FBP)\n+ EC5\n+ Strategy Pipeline (all)\n+ SLR EN1 (FBP)\n+ Profit Adjustment (FBP)',
  'NP05b': 'NP base zoning\n+ EC1\n+ DBP exogenous data\n+ zoning mods (FBP)\n+ H2 (FBP)\n+ H4 (FBP)\n+ H5 (FBP)\n+ EC5\n+ Strategy Pipeline (all)\n+ SLR EN1 (FBP)\n+ Profit Adjustment (FBP)',
  'NP05c': 

In [317]:
# these two should ideally align
pd.concat([pd.Series(table_for_asana_newline),
pd.Series(autodesc).apply(pd.Series).stack().reset_index(0,drop=True)],axis=1,keys=['manual','auto'])

,manual,auto
NP00,NP \n+ EC1,EC1
NP01,NP \n+ zoningmods (using FBP GG parcel assignm...,NP base zoning\n+ FBP exogenous data\n+ zoning...
NP02,NP \n+ zoningmods \n+ EC1 \n+ other housing st...,NP base zoning\n+ EC1\n+ FBP exogenous data\n+...
NP03,NP \n+ zoningmods \n+ EC1 \n+ other housing st...,NP base zoning\n+ EC1\n+ FBP exogenous data\n+...
NP04,NP \n+ zoningmods \n+ EC1 \n+ other housing st...,NP base zoning\n+ EC1\n+ FBP exogenous data\n+...
NP05,NP \n+ zoningmods \n+ EC1 \n+ other housing st...,NP base zoning\n+ EC1\n+ FBP exogenous data\n+...
NP05b,NP \n+ DBP Exog \n+ zoningmods \n+ EC1 \n+ oth...,NP base zoning\n+ EC1\n+ DBP exogenous data\n+...
NP05c,NP \n+ DBP base zoning \n+ zoningmods \n+ EC1 ...,DBP base zoning\n+ EC1\n+ FBP exogenous data\n...
NP05d,NP \n+ FBP pipeline \n+ zoningmods \n+ EC1 \n+...,NP base zoning\n+ EC1\n+ FBP exogenous data\n+...
FBP00,DBP \n+ exogenous of FBP,FBP base zoning\n+ FBP exogenous data\n+ Profi...


# Prepare command line args for running all diffs

Let's use the structure to generate commands for bash for easy construction of the diff commands, comparing a contrast run with its base reference.

In [318]:
cmd = 'python compare_two_runs_yaml_setup_files.py'


In [319]:
# for fn in list(output_base_path.glob('*.yaml')):
#     if fn.match('*test_FBP*'):
#         ref_path = dbp_input_path
#     else:
#         ref_path = np_input_path

#     out_cmd = f'{cmd} -p1 "{fn}" -p2 "{ref_path}"'
#     print(out_cmd)